# 线程同步 - 实践篇

本notebook通过实际代码演示线程同步的重要性和正确用法。

**学习目标：**
- 理解竞争条件的问题
- 掌握 `__syncthreads()` 的正确使用
- 学习并行规约算法


In [ ]:
%load_ext nvcc4jupyter


## 1. 没有同步的问题

演示在共享内存中缺少同步导致的错误结果。


In [ ]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

// 错误示例：没有同步
__global__ void noSync(int* output) {
    __shared__ int sdata[256];
    
    int tid = threadIdx.x;
    
    // 每个线程写入自己的位置
    sdata[tid] = tid;
    
    // ❌ 没有同步！
    // 可能读到未初始化的数据
    
    // 尝试读取下一个线程的值
    int neighbor = sdata[(tid + 1) % 256];
    
    output[tid] = neighbor;
}

// 正确示例：使用__syncthreads()
__global__ void withSync(int* output) {
    __shared__ int sdata[256];
    
    int tid = threadIdx.x;
    
    // 每个线程写入自己的位置
    sdata[tid] = tid;
    
    __syncthreads();  // ✓ 等待所有线程完成写入
    
    // 现在可以安全读取任意位置
    int neighbor = sdata[(tid + 1) % 256];
    
    output[tid] = neighbor;
}

int main() {
    printf("==========================================\n");
    printf("      同步 vs 无同步 对比演示\n");
    printf("==========================================\n\n");
    
    int n = 256;
    int *d_output, *h_output;
    h_output = (int*)malloc(n * sizeof(int));
    cudaMalloc(&d_output, n * sizeof(int));
    
    // 测试无同步版本（多次运行可能看到不同结果）
    printf("无同步版本 (可能有错误):\n");
    int errorCount = 0;
    for (int run = 0; run < 10; run++) {
        cudaMemset(d_output, 0, n * sizeof(int));
        noSync<<<1, 256>>>(d_output);
        cudaMemcpy(h_output, d_output, n * sizeof(int), cudaMemcpyDeviceToHost);
        
        // 检查结果
        int errors = 0;
        for (int i = 0; i < n; i++) {
            int expected = (i + 1) % 256;
            if (h_output[i] != expected) errors++;
        }
        if (errors > 0) errorCount++;
    }
    printf("  10次运行中出错次数: %d\n\n", errorCount);
    
    // 测试有同步版本
    printf("有同步版本 (总是正确):\n");
    withSync<<<1, 256>>>(d_output);
    cudaMemcpy(h_output, d_output, n * sizeof(int), cudaMemcpyDeviceToHost);
    
    int correct = 1;
    for (int i = 0; i < n; i++) {
        int expected = (i + 1) % 256;
        if (h_output[i] != expected) {
            correct = 0;
            break;
        }
    }
    printf("  结果: %s\n", correct ? "正确 ✓" : "错误 ✗");
    printf("  示例: output[0]=%d (期望1), output[255]=%d (期望0)\n", 
           h_output[0], h_output[255]);
    
    cudaFree(d_output);
    free(h_output);
    
    return 0;
}


## 2. 并行规约（Reduction）

并行求和是同步使用的经典示例。


In [ ]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

#define BLOCK_SIZE 256

// 并行规约求和
__global__ void reduce(float* input, float* output, int n) {
    __shared__ float sdata[BLOCK_SIZE];
    
    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + threadIdx.x;
    
    // 加载到共享内存
    sdata[tid] = (gid < n) ? input[gid] : 0.0f;
    __syncthreads();
    
    // 规约过程
    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (tid < stride) {
            sdata[tid] += sdata[tid + stride];
        }
        __syncthreads();  // 每一步都需要同步！
    }
    
    // Block的结果
    if (tid == 0) {
        output[blockIdx.x] = sdata[0];
    }
}

int main() {
    printf("==========================================\n");
    printf("           并行规约求和\n");
    printf("==========================================\n\n");
    
    int n = 1024;
    size_t size = n * sizeof(float);
    
    float *h_input = (float*)malloc(size);
    float *h_output = (float*)malloc(size);
    
    // 初始化: 1+2+3+...+n = n*(n+1)/2
    float expected = 0.0f;
    for (int i = 0; i < n; i++) {
        h_input[i] = (float)(i + 1);
        expected += h_input[i];
    }
    
    float *d_input, *d_output;
    cudaMalloc(&d_input, size);
    cudaMalloc(&d_output, size);
    cudaMemcpy(d_input, h_input, size, cudaMemcpyHostToDevice);
    
    int numBlocks = (n + BLOCK_SIZE - 1) / BLOCK_SIZE;
    
    printf("数组大小: %d\n", n);
    printf("Block大小: %d\n", BLOCK_SIZE);
    printf("Block数量: %d\n\n", numBlocks);
    
    // 第一次规约
    reduce<<<numBlocks, BLOCK_SIZE>>>(d_input, d_output, n);
    
    // 如果有多个Block，需要继续规约
    while (numBlocks > 1) {
        int newNumBlocks = (numBlocks + BLOCK_SIZE - 1) / BLOCK_SIZE;
        reduce<<<newNumBlocks, BLOCK_SIZE>>>(d_output, d_output, numBlocks);
        numBlocks = newNumBlocks;
    }
    
    cudaMemcpy(h_output, d_output, sizeof(float), cudaMemcpyDeviceToHost);
    
    printf("期望结果: %.0f\n", expected);
    printf("GPU结果:  %.0f\n", h_output[0]);
    printf("正确性:   %s\n", (h_output[0] == expected) ? "✓ 正确" : "✗ 错误");
    
    printf("\n规约过程示意 (以8个元素为例):\n");
    printf("Step 0: [1][2][3][4][5][6][7][8]\n");
    printf("Step 1: [3]   [7]   [11]  [15]   (stride=4)\n");
    printf("Step 2: [10]        [26]         (stride=2)\n");
    printf("Step 3: [36]                     (stride=1)\n");
    
    cudaFree(d_input);
    cudaFree(d_output);
    free(h_input);
    free(h_output);
    
    return 0;
}


## 3. 原子操作

当多个线程需要更新同一内存位置时，使用原子操作。


In [ ]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

// 错误：竞争条件
__global__ void incrementBad(int* counter) {
    (*counter)++;  // 读-改-写不是原子的
}

// 正确：使用原子操作
__global__ void incrementGood(int* counter) {
    atomicAdd(counter, 1);  // 原子操作
}

int main() {
    printf("==========================================\n");
    printf("           原子操作演示\n");
    printf("==========================================\n\n");
    
    int *d_counter;
    int h_counter;
    cudaMalloc(&d_counter, sizeof(int));
    
    int numThreads = 1024 * 1024;  // 100万个线程
    int threadsPerBlock = 256;
    int numBlocks = numThreads / threadsPerBlock;
    
    printf("线程总数: %d\n\n", numThreads);
    
    // 测试非原子版本
    cudaMemset(d_counter, 0, sizeof(int));
    incrementBad<<<numBlocks, threadsPerBlock>>>(d_counter);
    cudaMemcpy(&h_counter, d_counter, sizeof(int), cudaMemcpyDeviceToHost);
    printf("非原子操作结果: %d (期望: %d)\n", h_counter, numThreads);
    
    // 测试原子版本
    cudaMemset(d_counter, 0, sizeof(int));
    incrementGood<<<numBlocks, threadsPerBlock>>>(d_counter);
    cudaMemcpy(&h_counter, d_counter, sizeof(int), cudaMemcpyDeviceToHost);
    printf("原子操作结果:   %d (期望: %d)\n", h_counter, numThreads);
    
    printf("\n结论: 原子操作确保结果正确，但性能较低\n");
    
    cudaFree(d_counter);
    return 0;
}


## 总结

**关键要点：**

1. **`__syncthreads()`**
   - Block内所有线程必须执行
   - 用于共享内存读写同步
   - 不能放在条件分支中

2. **同步时机**
   - 写入共享内存后
   - 读取其他线程写入的数据前
   - 规约的每一步

3. **原子操作**
   - 用于多线程更新同一位置
   - 性能较低，尽量减少使用

## 练习

1. 实现并行求最大值（类似规约）
2. 使用共享内存优化的直方图计算
3. 尝试去掉规约中的`__syncthreads()`，观察结果
